In [111]:
from datasets import load_dataset, concatenate_datasets, DatasetDict, Dataset
import random

In [89]:
def format_choice(choice):
    choice = choice.replace("A)", "A.").replace("B)", "B.").replace("C)", "C.").replace("D)", "D.").replace("E)", "E.")
    return choice

# aquarat

In [90]:
aquarat = load_dataset("deepmind/aqua_rat")
aquarat_validation = aquarat["validation"]
aquarat_train = aquarat["train"]
aquarat_test = aquarat["test"]
def create_questions(question, options, correct_answer):
    formatted_question = question
    for i, option in enumerate(options):
        option = format_choice(option)
        formatted_question += f"\n{option}"
    

    index = ord(correct_answer) - ord("A")
    answer_text = options[index]
    answer_text = format_choice(answer_text)
    return formatted_question, answer_text

aquarat_validation = aquarat_validation.map(
    lambda x: {
        "question": create_questions(x["question"], x["options"], x["correct"])[0],
        "answer": create_questions(x["question"], x["options"], x["correct"])[1],
        "source": "deepmind/aqua_rat",
    },
    remove_columns=aquarat_validation.column_names,
)

aquarat_test = aquarat_test.map(
    lambda x: {
        "question": create_questions(x["question"], x["options"], x["correct"])[0],
        "answer": create_questions(x["question"], x["options"], x["correct"])[1],
        "source": "deepmind/aqua_rat",
    },
    remove_columns=aquarat_test.column_names,
)
aquarat_train = aquarat_train.map(
    lambda x: { 
        "question": create_questions(x["question"], x["options"], x["correct"])[0],
        "answer": create_questions(x["question"], x["options"], x["correct"])[1],
        "source": "deepmind/aqua_rat",
    },
    remove_columns=aquarat_train.column_names,
)

Map:   0%|          | 0/97467 [00:00<?, ? examples/s]

# Arc

In [91]:
arc_chal_val = load_dataset("allenai/ai2_arc", "ARC-Challenge")["validation"]
arc_chal_test = load_dataset("allenai/ai2_arc", "ARC-Challenge")["test"]
arc_easy_val = load_dataset("allenai/ai2_arc", "ARC-Easy")["validation"]
arc_eady_test = load_dataset("allenai/ai2_arc", "ARC-Easy")["test"]
arc_chal_train = load_dataset("allenai/ai2_arc", "ARC-Challenge")["train"]
def create_questions(question, options, answer_letter):
    formatted_question = question
    for i, option in enumerate(options["text"]):
        formatted_question += f"\n{chr(65 + i)}. {option}"
    
    # Get the correct answer option (A, B, C, etc.)
    answers = options["label"]
    index = answers.index(answer_letter)
    answer_text = f"{answer_letter}. {options['text'][index]}"
    
    return formatted_question, answer_text

arc_chal_val = arc_chal_val.map(
    lambda x: {
        "question": create_questions(x["question"], x["choices"], x["answerKey"])[0],
        "answer": create_questions(x["question"], x["choices"], x["answerKey"])[1],
        "source": "allenai/ai2_arc_challenge",
    },
    remove_columns=["choices", "answerKey", "id"]
)
arc_chal_test = arc_chal_test.map(
    lambda x: {
        "question": create_questions(x["question"], x["choices"], x["answerKey"])[0],
        "answer": create_questions(x["question"], x["choices"], x["answerKey"])[1],
        "source": "allenai/ai2_arc_challenge",
    },
    remove_columns=["choices", "answerKey", "id"]   
)
arc_easy_val = arc_easy_val.map(
    lambda x: {
        "question": create_questions(x["question"], x["choices"], x["answerKey"])[0],
        "answer": create_questions(x["question"], x["choices"], x["answerKey"])[1],
        "source": "allenai/ai2_arc_easy",
    },
    remove_columns=["choices", "answerKey", "id"]
)
arc_easy_test = arc_eady_test.map(
    lambda x: {
        "question": create_questions(x["question"], x["choices"], x["answerKey"])[0],
        "answer": create_questions(x["question"], x["choices"], x["answerKey"])[1],
        "source": "allenai/ai2_arc_easy",
    },
    remove_columns=["choices", "answerKey", "id"]
)
arc_chal_train = arc_chal_train.map(
    lambda x: {
        "question": create_questions(x["question"], x["choices"], x["answerKey"])[0],
        "answer": create_questions(x["question"], x["choices"], x["answerKey"])[1],
        "source": "allenai/ai2_arc_challenge",
    },
    remove_columns=["choices", "answerKey", "id"]
)


Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

# Med

In [105]:
med_val = load_dataset("openlifescienceai/medmcqa")["validation"]
med_train = load_dataset("openlifescienceai/medmcqa")["train"]
med_val = med_val.filter(lambda x: x["choice_type"] == "single")
med_train = med_train.filter(lambda x: x["choice_type"] == "single")


In [106]:
def create_question_med(question, options, correct_index, shuffle=True):
    if options is None or len(options) == 0:
        return question
    if shuffle:
        random.shuffle(options)
    correct_option = options[correct_index]
    correct_answer = f"{chr(65 + correct_index)}. {correct_option}"
    for i, option in enumerate(options):
        question += f"\n{chr(65 + i)}. {option}"
    return question, correct_answer

med_val = med_val.map(
    lambda x: {
        "question": (created_question := create_question_med(
            x["question"], 
            [x["opa"], x["opb"], x["opc"], x["opd"]], 
            x["cop"]))[0],
        "answer": created_question[1],
        "source": "openlifescienceai/medmcqa",
    },
    remove_columns=[col for col in med_val.column_names if col not in ["question", "answer", "source"]]
)
med_train = med_train.map(
    lambda x: {
        "question": (created_question := create_question_med(
            x["question"], 
            [x["opa"], x["opb"], x["opc"], x["opd"]], 
            x["cop"]))[0],
        "answer": created_question[1],
        "source": "openlifescienceai/medmcqa",
    },
    remove_columns=[col for col in med_train.column_names if col not in ["question", "answer", "source"]]
)


Map:   0%|          | 0/120765 [00:00<?, ? examples/s]

# Sciq

In [107]:
dataset = load_dataset("allenai/sciq")
sciq_test = dataset["test"]
sciq_val = dataset["validation"]
sciq_train = dataset["train"]
def create_question(question, options, shuffle=True):
    if options is None or len(options) == 0:
        return question
    correct_option = options[0]
    if shuffle:
        random.shuffle(options)
    correct_index = options.index(correct_option)
    correct_answer = f"{chr(65 + correct_index)}. {correct_option}"
    for i, option in enumerate(options):
        question += f"\n{chr(65 + i)}. {option}"
    return question, correct_answer

# Map the dataset to only keep two columns: question and correct answer
sciq_val = sciq_val.map(
    lambda x: 
    {
        "question": (created_question := create_question(x["question"], [x["correct_answer"], x["distractor1"], x["distractor2"], x["distractor3"]], shuffle=True))[0],
        "answer": created_question[1],
        "source": "allenai/sciq",
    },
    remove_columns=sciq_val.column_names
)

sciq_test = sciq_test.map(
    lambda x: 
    {
        "question": (created_question := create_question(x["question"], [x["correct_answer"], x["distractor1"], x["distractor2"], x["distractor3"]], shuffle=True))[0],
        "answer": created_question[1],
        "source": "allenai/sciq",
    },
    remove_columns=sciq_test.column_names
)
sciq_train = sciq_train.map(
    lambda x: 
    {
        "question": (created_question := create_question(x["question"], [x["correct_answer"], x["distractor1"], x["distractor2"], x["distractor3"]], shuffle=True))[0],
        "answer": created_question[1],
        "source": "allenai/sciq",
    },
    remove_columns=sciq_train.column_names
)
sciq_val


Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 1000
})

# MMLU

In [108]:
mmlu = load_dataset("cais/mmlu", "all")
mmlu_stem_subjects = [
    "abstract_algebra",
    "anatomy",
    "college_biology",
    "college_chemistry",
    "college_computer_science",
    "college_mathematics",
    "college_physics",
    "electrical_engineering",
    "elementary_mathematics",
    "high_school_biology",
    "high_school_chemistry",
    "high_school_computer_science",
    "high_school_mathematics",
    "high_school_physics",
    "high_school_statistics"
]
mmlu_train = mmlu["auxiliary_train"]
mmlu = mmlu.filter(lambda x: x["subject"] in mmlu_stem_subjects)
mmlu_val = mmlu["validation"]
mmlu_test = mmlu["test"]

def create_mmlu_question(question, options, index):
    formatted_question = question
    for i, option in enumerate(options):
        formatted_question += f"\n{chr(65 + i)}. {option}"
    
    # Get the correct answer option (A, B, C, etc.)
    answer_letter = chr(65 + index)  
    answer_text = f"{answer_letter}. {options[index]}"
    
    return formatted_question, answer_text


mmlu_val = mmlu_val.map(
    lambda x: {
        "question": create_mmlu_question(x["question"], x["choices"], x["answer"])[0],
        "response": create_mmlu_question(x["question"], x["choices"], x["answer"])[1],
        "source": "cais/mmlu",
    },
    remove_columns=mmlu_val.column_names
)

mmlu_test = mmlu_test.map(
    lambda x: {
        "question": create_mmlu_question(x["question"], x["choices"], x["answer"])[0],
        "response": create_mmlu_question(x["question"], x["choices"], x["answer"])[1],
        "source": "cais/mmlu",
    },
    remove_columns=mmlu_test.column_names
)
mmlu_train = mmlu_train.map(
    lambda x: {
        "question": create_mmlu_question(x["question"], x["choices"], x["answer"])[0],
        "response": create_mmlu_question(x["question"], x["choices"], x["answer"])[1],
        "source": "cais/mmlu",
    },
    remove_columns=mmlu_train.column_names
)
mmlu_train = mmlu_train.rename_column("response", "answer")
mmlu_val = mmlu_val.rename_column("response", "answer")
mmlu_test = mmlu_test.rename_column("response", "answer")
mmlu_val

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 271
})

In [109]:
mmlu_test

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 2554
})

# M1 prefrences

In [ ]:
import json
with open("../../data/m1_preference_data.json") as f:
    m1_data = json.load(f)

dataset = {
    "question": [],
    "answer": [],
    "source": []
}

for qst in m1_data:
    if qst["question_type"] == "mcq":
        question_body = qst["question_body"]
        options = qst["question_options"]
        correct_answer = qst["question_answer"]
        if correct_answer[0] == "[":
            correct_answer = eval(correct_answer)
            correct_answer = correct_answer[0]
            options = [opt for opt in options if opt not in correct_answer]
        if isinstance(correct_answer, int):
            correct_answer = options[correct_answer -1]
        if isinstance(correct_answer, str) and correct_answer.isdigit():
            if int(correct_answer) > 0 and int(correct_answer) <= len(options):
                correct_answer = options[int(correct_answer) - 1]
        for i, option in enumerate(options):
            if correct_answer in option:
                correct_answer = chr(65 + i) + ". " + option
                question = question_body + "\n" + "\n".join([chr(65 + i) + ". " + opt for i, opt in enumerate(options)])
                dataset["question"].append(question)
                dataset["answer"].append(correct_answer)
                dataset["source"].append("m1_preference_data")
                break


len(dataset["question"]), len(dataset["answer"]), len(dataset["source"])
dataset_formatted = Dataset.from_dict(dataset)
m1_dataset = dataset_formatted.map(
    lambda x: {
        "question": x["question"],
        "answer": x["answer"],
        "source": x["source"],
    },
    remove_columns=["question", "answer", "source"]
)
print(m1_dataset[0])  # Display the first formatted question and answer
m1_dataset

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

{'question': 'In JOS, suppose a value is passed between two Envs. What is the minimum number of executed system calls?\nA. 1\nB. 3\nC. 4', 'answer': 'B. 3', 'source': 'm1_preference_data'}


Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 450
})

# MMLU-PRO

In [113]:
mmlu_pro_test = load_dataset("TIGER-Lab/MMLU-Pro")["test"]
mmlu_pro_val = load_dataset("TIGER-Lab/MMLU-Pro")["validation"]

def create_mmlu_pro_question(question, options, index):
    letters = [chr(65 + i) for i in range(len(options))]
    formatted_question = question
    for i, option in enumerate(options):
        formatted_question += f"\n{letters[i]}. {option}"
    answer_letter = letters[index]
    answer_text = f"{answer_letter}. {options[index]}"
    return formatted_question, answer_text

mmlu_pro_val = mmlu_pro_val.map(
    lambda x: {
        "question": create_mmlu_pro_question(x["question"], x["options"], x["answer_index"])[0],
        "answer": create_mmlu_pro_question(x["question"], x["options"], x["answer_index"])[1],
        "source": "TIGER-Lab/MMLU-Pro",
    },
    remove_columns=mmlu_pro_val.column_names
)
mmlu_pro_test = mmlu_pro_test.map(
    lambda x: {
        "question": create_mmlu_pro_question(x["question"], x["options"], x["answer_index"])[0],
        "answer": create_mmlu_pro_question(x["question"], x["options"], x["answer_index"])[1],
        "source": "TIGER-Lab/MMLU-Pro",
    },
    remove_columns=mmlu_pro_test.column_names
)

mmlu_pro_val

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 70
})

In [125]:
for ds in [    aquarat_train,
    arc_chal_train,
    med_train,
    sciq_train,
    mmlu_train]:
    print(ds)

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 97467
})
Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 1119
})
Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 120765
})
Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 11679
})
Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 99842
})


# Merge all

In [129]:
validation_dataset = concatenate_datasets([
    aquarat_validation,
    arc_chal_val,
    arc_easy_val,
    med_val,
    sciq_val,
    mmlu_val,
    mmlu_pro_val
])

test_dataset = concatenate_datasets([
    aquarat_test,
    arc_chal_test,
    arc_easy_test,
    sciq_test,
    mmlu_test,
    mmlu_pro_test
])

train_dataset = concatenate_datasets([
    aquarat_train,
    arc_chal_train,
    med_train,
    sciq_train,
    mmlu_train,
    m1_dataset
])
validation_dataset = validation_dataset.shuffle(seed=42)
test_dataset = test_dataset.shuffle(seed=42)
train_dataset = train_dataset.shuffle(seed=42)

In [130]:
validation_dataset

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 5280
})

In [131]:
test_dataset

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 19388
})

In [132]:
train_dataset

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 331322
})

In [138]:
import json
with open("MCQ_prompts.json") as f:
    mcq_prompts = json.load(f)

def add_prompts(example):
    prompt = random.choice(mcq_prompts)
    question = prompt.replace("<question>", example["question"])
    example["question"] = question
    return example

final_train_dataset = train_dataset.map(add_prompts)
final_validation_dataset = validation_dataset.map(add_prompts)
final_test_dataset = test_dataset.map(add_prompts)
final_train_dataset


Map:   0%|          | 0/5280 [00:00<?, ? examples/s]

Map:   0%|          | 0/19388 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'source'],
    num_rows: 331322
})

In [139]:
complete_dataset = DatasetDict({
    "train": final_train_dataset,
    "validation": final_validation_dataset,
    "test": final_test_dataset,
})
complete_dataset.push_to_hub("HAissa/MCQ_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/332 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HAissa/MCQ_dataset/commit/a4a0d29bfbd858f41959fc2e5368224ca26ea482', commit_message='Upload dataset', commit_description='', oid='a4a0d29bfbd858f41959fc2e5368224ca26ea482', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/HAissa/MCQ_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='HAissa/MCQ_dataset'), pr_revision=None, pr_num=None)